# Mentioned facts in the paper

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pylab as plt
import time
from copy import deepcopy

import importlib
import matplotlib as mpl
import matplotlib.patches as mp
import pandas as pd

from pyloric import create_prior, simulate, summary_stats
from stg_energy.common import col, svg, get_labels_8pt
import stg_energy.fig1_posterior.viz as viz
from sbi.utils import pairplot
from stg_energy import check_if_close_to_obs 

In [3]:
pd.options.display.max_columns=999

### Almost no prior simulations match our criteria

In [21]:
theta = pd.read_pickle("../results/simulation_data_Tube_MLslurm_cluster/01_simulate_11deg/data/valid_circuit_parameters.pkl")
x = pd.read_pickle("../results/simulation_data_Tube_MLslurm_cluster/01_simulate_11deg/data/valid_simulation_outputs.pkl")
seeds = np.load("../results/simulation_data_Tube_MLslurm_cluster/01_simulate_11deg/data/valid_seeds.npy")

theta_np = theta.to_numpy()
x_np = x.to_numpy()

In [18]:
close_sim = check_if_close_to_obs(x_np)

In [22]:
print("Number of prior simulations that fulfilled our criteria:  ", np.sum(close_sim))
print("Overall, we had simulated 3 million times (dataset `/01_simulate_11deg/data/all_circuit_parameters.pkl`)")

Number of prior simulations that fulfilled our criteria:   0
Overall, we had simulated 3 million times (dataset `/01_simulate_11deg/data/all_circuit_parameters.pkl`)
